In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import random

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



In [2]:
df = pd.read_csv("./daily-min-temperatures.csv")
df["year"] = [0]*len(df["Date"])
df["month"] = [0]*len(df["Date"])
df["day"] = [0]*len(df["Date"])
for i in range(len(df["year"])):
    df["year"][i] = int(df["Date"][i][0:4])
    df["month"][i] = int(df["Date"][i][5:7])
    df["day"][i] = int(df["Date"][i][8:])

<ipython-input-2-e02ed463d761>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["year"][i] = int(df["Date"][i][0:4])
<ipython-input-2-e02ed463d761>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["month"][i] = int(df["Date"][i][5:7])
<ipython-input-2-e02ed463d761>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["day"][i] = int(df["Date"][i][8:])


In [3]:
df

,Date,Temp,year,month,day
0,1981-01-01,20.7,1981,1,1
1,1981-01-02,17.9,1981,1,2
2,1981-01-03,18.8,1981,1,3
3,1981-01-04,14.6,1981,1,4
4,1981-01-05,15.8,1981,1,5
...,...,...,...,...,...
3645,1990-12-27,14.0,1990,12,27
3646,1990-12-28,13.6,1990,12,28
3647,1990-12-29,13.5,1990,12,29
3648,1990-12-30,15.7,1990,12,30


In [4]:
train, test = train_test_split(df, test_size= 0.2, shuffle=False)
train = train[["year", 'month', "day", "Temp"]].reset_index(drop=True)
test = test[["year", 'month', "day", "Temp"]].reset_index(drop=True)

x_train = train[["year", 'month', "day"]].values.reshape((train[["year", 'month', "day"]].values.shape[0], 1, train[["year", 'month', "day"]].values.shape[1]))
y_train = train["Temp"].values

x_test = test[["year", 'month', "day"]].values.reshape((test[["year", 'month', "day"]].values.shape[0], 1, test[["year", 'month', "day"]].values.shape[1]))
y_test = test["Temp"].values

In [5]:
train

,year,month,day,Temp
0,1981,1,1,20.7
1,1981,1,2,17.9
2,1981,1,3,18.8
3,1981,1,4,14.6
4,1981,1,5,15.8
...,...,...,...,...
2915,1988,12,26,9.5
2916,1988,12,27,12.9
2917,1988,12,28,12.9
2918,1988,12,29,14.8


In [6]:
#from tensorflow tutorial
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
               train_df=train[["month", "day", "Temp"]], val_df=test[["month", "day", "Temp"]], test_df=test[["month", "day","Temp"]],
               label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels


    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
        # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
        # And cache it for next time
            self._example = result
        return result

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=True,
          batch_size=32,)

        ds = ds.map(self.split_window)

        return ds
#     WindowGenerator.split_window = split_window
#     WindowGenerator.make_dataset = make_dataset
#     WindowGenerator.train = train
#     WindowGenerator.val = val
#     WindowGenerator.test = test
#     WindowGenerator.example = example

In [7]:
k = 14
wide_window = WindowGenerator(
    input_width=k, label_width=k, shift=1,
    label_columns=['Temp'])
wide_window

Total window size: 15
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Label indices: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Label column name(s): ['Temp']

In [8]:

lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(64, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

lstm_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=["mse", tf.metrics.MeanAbsoluteError()])



In [9]:
lstm_model.fit(wide_window.train, epochs=40, validation_data=wide_window.val)

Epoch 1/40
91/91 [==============================] - 2s 9ms/step - loss: 57.7103 - mse: 57.7103 - mean_absolute_error: 6.2106 - val_loss: 21.1458 - val_mse: 21.1458 - val_mean_absolute_error: 3.6787
Epoch 2/40
91/91 [==============================] - 1s 6ms/step - loss: 16.0628 - mse: 16.0628 - mean_absolute_error: 3.1177 - val_loss: 12.2099 - val_mse: 12.2099 - val_mean_absolute_error: 2.7090
Epoch 3/40
91/91 [==============================] - 1s 6ms/step - loss: 10.0856 - mse: 10.0856 - mean_absolute_error: 2.4237 - val_loss: 8.0381 - val_mse: 8.0381 - val_mean_absolute_error: 2.1641
Epoch 4/40
91/91 [==============================] - 1s 6ms/step - loss: 8.0352 - mse: 8.0352 - mean_absolute_error: 2.1746 - val_loss: 6.6913 - val_mse: 6.6913 - val_mean_absolute_error: 1.9980
Epoch 5/40
91/91 [==============================] - 1s 6ms/step - loss: 7.1635 - mse: 7.1635 - mean_absolute_error: 2.0627 - val_loss: 6.1049 - val_mse: 6.1049 - val_mean_absolute_error: 1.9252
Epoch 6/40
91/91 [==

In [10]:
lstm_model.evaluate(wide_window.test, verbose=0)

[4.777579307556152, 4.777580261230469, 1.7370178699493408]